In [21]:
%load_ext autoreload 

import numpy as np
import pandas as pd
# from aocd i
# import get_data
import os
import re
import time
import networkx as nx
from heapq import *
import functools
import functools
from  math import log
from tqdm import tqdm
import sys


module_path=os.path.abspath(os.path.join('../'))
sys.path.append(module_path)
from ComplexMap import ComplexMap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def get_operation(oper_str):
    return oper_str.replace("AND","&").replace("XOR","^").replace("OR","|")

def build_calc_tree(rule_strs):
    rules = [get_gate(rule) for rule in rule_strs if len(rule)>0]
    edges = [[(child1,node),(child2,node)] for child1,_,child2,node in rules]
    edges = [x for y in edges for x in y]
    node_operations = {node:get_operation(oper) for _,oper,_,node in rules}

    G=nx.from_edgelist(edges,nx.DiGraph)
    
    return G, node_operations
                       
def get_gate(line):
    return re.match(r"(\w+)\W+(\w+)\W+(\w+)\W+(\w+)",line).groups()

def get_inputs(file):
    with open(file) as f:
        lines = f.readlines()
    lines = list(map(str,list(map(lambda x:x.strip(),lines))))

    i = 0
    while lines[i]: i+=1

    # initialisation part
    values_dict = {}
    for line in lines[:i]:
        gate, val = line.split(": ") 
        values_dict[gate]=bool(int(val))

    # rules part
    rules = lines[i+1:]
    G,node_operations = build_calc_tree(rules)
    
    return values_dict,G,node_operations

def get_targets(G):
    return [n for n in G.nodes if len(nx.descendants(G,n))==0]

def get_in_nodes(G):
    return [n for n in G.nodes if len(nx.ancestors(G,n))==0]
    
# get_inputs("test.txt")

In [23]:
val_dict, G,nodes_ops = get_inputs("test.txt")


In [24]:
def evaluate_node(G,n,values_calced,nodes_ops):
    if not n in values_calced: 
        # print(f"n={n}")
        ch1,ch2 = G.predecessors(n)
        # print("  ",n,"::",ch1,ch2)
        ch1_val = evaluate_node(G,ch1,values_calced,nodes_ops)
        ch2_val = evaluate_node(G,ch2,values_calced,nodes_ops)
        values_calced[n] = eval(" ".join([str(ch1_val),nodes_ops[n],str(ch2_val)]))

    return values_calced[n]    

In [25]:
assert evaluate_node(G,"z02",val_dict,nodes_ops)
assert not evaluate_node(G,"z01",val_dict,nodes_ops)
assert not evaluate_node(G,"z00",val_dict,nodes_ops)
print(val_dict)

{'x00': True, 'x01': True, 'x02': True, 'y00': False, 'y01': True, 'y02': False, 'z02': True, 'z01': False, 'z00': False}


In [26]:
def partA(file):
    val_dict, G, nodes_ops = get_inputs(file)

    targets=get_targets(G)
    targets.sort(key=lambda x:int(x[1:]))

    res = [str(int(evaluate_node(G,t,val_dict,nodes_ops))) for t in targets[::-1]]
    return int("".join(res),2)
    

In [27]:
assert partA("test.txt") == 4
assert partA("test2.txt") == 2024
assert partA("input.txt") == 60614602965288
print("All ok")

All ok


In [28]:
from itertools import product

def getNum(val_dict,prefix):
    """ derive a decimal number from the "x", "y" or "z" nodes depending on the prefix"""
    res = [(key[1:],val) for key,val in val_dict.items() if key.startswith(prefix)]
    res.sort(key=lambda x:int(x[0]),reverse=True)
    res = "".join([str(x) for _,x in res])
    return int(res,2),res

In [29]:
def swap_nodes(G,x,y,nodes_ops):
    # print("Swapping nodes",x,y)
    G_copy = G.copy()
    edges_old = [(z,x) for z in G_copy.predecessors(x)] + [(z,y) for z in G_copy.predecessors(y)]
    edges_new = [(z,y) for z in G_copy.predecessors(x)] + [(z,x) for z in G_copy.predecessors(y)]
        
    if (y,y) in edges_new or (x,x) in edges_new:
        return G,nodes_ops,False
    else:
        # print("Removing old edges",edges_old)  
        G_copy.remove_edges_from(edges_old)
        G_copy.add_edges_from(edges_new)
        if list(nx.simple_cycles(G_copy)):
            # "INTRODUCED GRAPH!!!")
            return G,nodes_ops,False
            
        nodes_ops_copy = nodes_ops.copy()
        x_val = nodes_ops[x]
        y_val = nodes_ops[y]
        nodes_ops_copy[x]=y_val
        nodes_ops_copy[y]=x_val
    
    return G_copy, nodes_ops_copy,True


In [30]:
readable_oper = {"^":"XOR","&":"AND","|":"OR"}
import time
def print_calcgraph(G,nodes_ops,to_file=False):
    out=[]
    for n in G.nodes:
        # print(".",end="")
        if len(list(G.predecessors(n)))>0:
            ch1,ch2 = G.predecessors(n)
            out+=[f"{ch1} {readable_oper[nodes_ops[n]]} {ch2} -> {n}"]
    res = "\n".join(out)
    if to_file:
        with open(f"interm_out_{time.time()}.txt","w") as f:
            f.write(res)
    else:
        print(res)
            

In [31]:
def test_the_target(G_sub,nodes_ops,z,leaves_to_test,max_bit=45,verbose=False):
        
    val_dict = {pref + f"{i:02d}":0 for i in range(max_bit) for pref in {"x","y"}}
    
    zero_digits =len(val_dict)//2
    
    B = {0,1}
    
    combinations = list(product(B,repeat=len(leaves_to_test)))
    for entry in combinations:
        val_dict_copy = val_dict.copy()
        for i,val in enumerate(entry): val_dict_copy[leaves_to_test[i]]=val            
        
        res = evaluate_node(G_sub,z,val_dict_copy,nodes_ops)
        
        xNum,xNum_str = getNum(val_dict_copy,"x")
        yNum,yNum_str = getNum(val_dict_copy,"y")

        z_idx = int(z[1:])
        zExp = xNum + yNum
        zExp_str = bin(zExp)[2:]
        # print(zExp,bin(zExp),zExp_str,(z_idx+1-len(zExp_str)))
        
        zExp_str = "0"*max(z_idx+1-len(zExp_str),0)+zExp_str
        zExp_str = zExp_str[::-1]
        
        zExp_digit = int(zExp_str[z_idx])
        zNum_digit = int(val_dict_copy[z])
        if zExp_digit != zNum_digit: 

            if verbose:
                print(f"Found one: {z}")
                
            return False

    return True

In [41]:
def fix_nodes(G,nodes_ops,intersect,tgts,leaves_to_test):
    for n1 in intersect:
        for n2 in intersect:     
            if n1 == n2: continue

            # print(f"  trying swap {n1},{n2}")
            G_copy,nodes_ops_copy, success = swap_nodes(G,n1,n2,nodes_ops)
            if not success: continue

            # print(f"    - swapped {n1},{n2}")
            works = True
            for z in tgts:
                # print(f"    testing {z}") 
                if not test_the_target(G_copy,nodes_ops_copy,z,leaves_to_test,verbose=False):
                    works = False
                    break
            if works:
                print("Substitution",n1,n2)
                return G_copy, nodes_ops_copy, [(n1,n2)]
    return G, nodes_ops, []

In [42]:
def flatten_set(nested_collection):
    return {x for y in nested_collection for x in y}

def check_and_fix(G,nodes_ops,targets):
    substitutions = []
    
    for tgt in targets:
        idx = int(tgt[1:])
        idx_str = f"{idx:02d}"
        idx_prev_str = f"{idx-1:02d}"
        idx_next_str = f"{idx+1:02d}"
        tgt_next = "z"+idx_next_str
        if "x"+idx_prev_str not in G.nodes: continue
        if tgt_next not in G.nodes: continue

        leaves = [pref + suff for pref in {"x","y"} for suff in {idx_prev_str,idx_str, idx_next_str}]
        leaves = [x for x in leaves if x in G.nodes]
        successors = flatten_set([nx.descendants(G,n) for n in leaves])
        ancestors = set(list(nx.ancestors(G,tgt)) + list(nx.ancestors(G,tgt_next))+[tgt, tgt_next])
        intersect = successors.intersection(successors,ancestors).difference(leaves)

        good_node = test_the_target(G,nodes_ops,tgt,leaves)
        
        if not good_node: 
            print("Bad node",tgt)
            G,nodes_ops, subst = fix_nodes(G,nodes_ops,intersect,[tgt,"z"+idx_next_str],leaves)
            
            substitutions+= subst
    return substitutions,G

In [43]:
def partB(file):
    val_dict, G, nodes_ops = get_inputs(file)

    targets=get_targets(G)
    targets.sort(key=lambda x:int(x[1:]))

    res, G = check_and_fix(G,nodes_ops, targets)
    res_str = ",".join(sorted([x for (x,y) in res]+[y for x,y in res]))
    return res_str, G
    

In [44]:
res,G = partB("input.txt")
print(f"Part B: {res}")

Bad node z06
Substitution z06 hwk
Bad node z25
Substitution qmd tnt
Bad node z31
Substitution z31 hpc
Bad node z37
Substitution cgr z37
Part B: cgr,hpc,hwk,qmd,tnt,z06,z31,z37


In [36]:
res

'cgr,hpc,hwk,qmd,tnt,z06,z31,z37'

In [45]:
assert res=="cgr,hpc,hwk,qmd,tnt,z06,z31,z37"
print("OK answer part B")

OK answer part B
